In [1]:
from zipfile import ZipFile
from estnltk.converters import json_to_text


def iter_analysed_zip(file):
    with ZipFile(file) as inzip:
        namelist = inzip.namelist()
        for name in namelist:
            if not name.endswith('.json'):
                continue
            with inzip.open(name) as infile:
                yield name, json_to_text(infile.read().decode())

In [2]:
from estnltk.storage import PostgresStorage
from collections import OrderedDict


# ~/.pgpass file format: hostname:port:database:username:password
storage = PostgresStorage(schema='public', pgpass_file='~/.pgpass')

meta = OrderedDict([('file', 'str'),
                    ('subcorpus', 'str'),
                    ('title', 'str'),
                    ('type', 'str')])

collection = storage.get_collection('analysed50', meta)
collection.create()

In [3]:
for file, text in iter_analysed_zip('koond_analysed_json_50.zip'):
    meta = {**text.meta, 'file': file}
    collection.insert(text, meta=meta)

In [4]:
storage.close()